In [1]:
%load_ext autoreload
%autoreload 2
import config
import torch
from app.Classifiers.Classifier_NN import Classifier_IMGNET, Classifier_MNIST
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from sklearn.model_selection import train_test_split
from app.NObSP.NObSP_Decomposition import NObSP_NN_single_MultiOutput, NObSP_NN_single_MultiOutput_reg
import torch.nn.functional as F
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader as tf_dataloader
import torchvision.transforms as transforms
import pandas as pd
import seaborn as sns
import os
from tqdm.notebook import tqdm
from torchvision.transforms.functional import to_pil_image
from matplotlib import colormaps
import PIL
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
# functions to show an image
def imshow(img):
    plt.figure(figsize=(2, 2))
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()

In [4]:
# Cargar los tensores guardados
X = torch.load('tensors.pth')
X = torch.stack(X) # Convierte 'X' en un tensor si es una lista de listas
X_small = torch.load('tensors_avgpooling.pth')
y = torch.load('labels.pth')
y = torch.tensor(y)  # Convierte 'y' en un tensor si es una lista de enteros
y = torch.nn.functional.one_hot(y, num_classes=1000)

In [5]:
X_sample = []
X_small_sample = []
y_sample = []
index_used = []

sample_size = 200
y_values = [148, 22, 21, 208, 291, 283, 2, 281, 292, 269]

for i in y_values:
    indices = torch.where(y.argmax(dim=1) == i)[0]
    indices = indices[torch.randperm(len(indices))[:sample_size]]
    index_used.append(indices)
    X_sample.append(X[indices])
    X_small_sample.append(X_small[indices])
    y_sample.append(y[indices])

# Verifica que cada tensor en X_sample y y_sample es diferente
for i in range(len(X_sample)):
    for j in range(i + 1, len(X_sample)):
        assert id(X_sample[i]) != id(X_sample[j])
        assert id(y_sample[i]) != id(y_sample[j])

index_used = [tensor.item() for sublist in index_used for tensor in sublist]
X_sample = torch.cat(X_sample)
y_sample = torch.cat(y_sample)
X_small_sample = torch.cat(X_small_sample)

In [6]:
from torchvision.models import resnet18 as model 
from torchvision.models import ResNet18_Weights as model_weights
# Cargar el modelo preentrenado de DenseNet121
weights = model_weights.IMAGENET1K_V1
model = model(weights=weights)
model.eval()  # Poner el modelo en modo de evaluación

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
X_sample.shape

torch.Size([2000, 25088])

In [8]:
class ClassifierBig(nn.Module):
    def __init__(self, model):
        super(ClassifierBig, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.weights = model.fc.weight.data
        self.classifier = nn.Linear(512, 1000)
        self.classifier.weight.data = self.weights

    def forward(self, x):
        x = x.view(-1, 512, 7, 7)
        x = self.avgpool(x)
        x_t = x.view(x.size(0), -1)  
        y_lin = self.classifier(x_t)
        prob = F.softmax(y_lin, dim=1)
        return prob, x_t, y_lin

classifier_big = ClassifierBig(model)

In [9]:
class ClassifierSmall(nn.Module):
    def __init__(self, model):
        super(ClassifierSmall, self).__init__()
        self.weights = model.fc.weight.data
        self.classifier = nn.Linear(512, 1000)
        self.classifier.weight.data = self.weights

    def forward(self, x):
        y_lin = self.classifier(x)
        prob = F.softmax(x, dim=1)
        return prob, x, y_lin

classifier_small = ClassifierSmall(model)

In [10]:
classifier_big.eval()  # Setting the model in evaluation mode
with torch.inference_mode():
    y_prob_1, x_trans_total_1, y_est_1 = classifier_big(
        X_sample
    )  # Computing the nonlinear transformation of the input data X
    
classifier_small.eval()  # Setting the model in evaluation mode
with torch.inference_mode():
    y_prob_2, x_trans_total_2, y_est_2 = classifier_small(
        X_small_sample
    )  # Computing the nonlinear transformation of the input data X

In [11]:
len(np.unique(np.argmax(F.softmax(y_est_1, dim=1), axis=1)))

204

In [12]:
if os.path.exists("betas_tensor_small.pth"):
    betas_tensor_small = torch.load("betas_tensor_small.pth")
else:
    betas_tensor_small = NObSP_NN_single_MultiOutput_reg(
        X_small_sample, y_est_2, classifier_small
    )  # Computing the decomposition using NObSP. The Alpha parameters are the weigths for the Interpretation Layer
    torch.save(betas_tensor_small, "betas_tensor_small.pth")

X:  torch.Size([2000, 512])
beta:  torch.Size([512, 512, 1000])
[  2   3   4   5   6  10  13  15  16  17  18  19  20  21  22  23  24  33
  36  48  49  65  73  81  83  85  86  87  89  91  92  93  95  98  99 103
 104 105 106 107 111 123 126 127 129 133 135 137 138 140 143 144 146 147
 148 149 150 154 155 159 160 162 163 164 165 167 169 170 172 173 174 175
 176 177 178 179 182 191 205 206 207 208 209 210 211 212 215 218 220 222
 223 224 225 227 234 235 236 237 243 244 245 249 256 257 259 260 261 262
 263 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284
 285 286 287 289 291 292 294 295 296 299 306 307 330 331 332 333 336 339
 341 342 343 344 347 350 351 352 355 358 359 366 369 371 373 385 388 390
 391 394 397 408 410 430 437 450 478 517 559 562 588 704 733 750 760 761
 770 801 814 831 833 842 851 858 863 871 897 900 913 917 920 971 976 977
 978 980 983 993 996]


Number of tensors:   0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
if os.path.exists("betas_tensor.pth"):
    betas_tensor = torch.load("betas_tensor.pth")
else:
    betas_tensor = NObSP_NN_single_MultiOutput_reg(
        X_sample, y_est_1, classifier_big
    )  # Computing the decomposition using NObSP. The Alpha parameters are the weigths for the Interpretation Layer
    torch.save(betas_tensor, "betas_tensor.pth")

X:  torch.Size([2000, 25088])
beta:  torch.Size([512, 25088, 1000])
[  2   3   4   5   6  10  13  15  16  17  18  19  20  21  22  23  24  33
  36  48  49  65  73  81  82  83  85  86  87  89  91  93  95  98  99 103
 104 105 106 107 111 123 126 127 129 133 135 137 138 140 143 144 146 147
 148 149 150 154 155 159 160 162 163 164 165 167 169 170 172 173 174 175
 176 177 178 179 182 191 205 206 207 208 209 210 211 212 215 218 220 222
 223 224 225 227 234 235 236 237 243 244 245 249 256 257 259 260 261 262
 263 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284
 285 286 287 289 291 292 293 294 295 296 299 306 307 330 331 332 333 336
 339 341 342 343 344 347 350 351 352 355 358 359 366 369 371 373 385 388
 390 391 394 397 408 410 430 437 450 478 517 559 562 588 704 733 750 760
 761 770 801 814 831 833 842 851 858 863 871 897 900 913 917 920 971 976
 977 978 980 983 993 996]


Number of tensors:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def get_contrib(X, model, betas):
    X_new = X.clone()
    clase = int(torch.argmax(model(X_new.unsqueeze(0))[0].detach(), axis=1))
    X_target_tot = torch.zeros((1, list(model.children())[-1].in_features))

    X_temp = X_new.repeat(len(X_new), 1)
    X_temp = torch.eye(len(X_new)) * X_temp
    with torch.inference_mode():
        _, X_latent, _ = model(X_new.unsqueeze(0))
        _, X_target, _ = model(X_temp)
    X_target_tot = torch.cat((X_target_tot, X_target), 0)
    X_target_tot = X_target_tot[1:]

    contrib = (X_target_tot @ betas[:, :, clase]).diag()
    return clase, contrib, X_latent

In [ ]:
def train_contribs(X, model, betas):
    contribs = []
    labels = []
    tensors = []
    latents = []
    for i in tqdm(range(len(X))):
        label, contrib, latent = get_contrib(X[i], model, betas)
        contribs.append(contrib.numpy())
        labels.append(label)
        tensors.append(X[i].numpy())
        latents.append(latent.numpy()[0])

    train_contribs = pd.DataFrame(
        {
            'Contrib': contribs,
            'Label': labels,
            'Flatten': tensors,
            'Latent': latents
        }
    )
    return train_contribs

In [ ]:
if os.path.exists("train_contribs.pkl"):
    train_contribs = pd.read_pickle("train_contribs.pkl")
else: 
    train_contribs = train_contribs(X_sample, classifier_big, betas_tensor)
    train_contribs.to_pickle("train_contribs.pkl")
    
train_contribs.head()

In [ ]:
def viz_contrib(train_contribs):
    
    tsne = TSNE(
        n_components=2,
        random_state=42,
        perplexity=20,
    )
    tsne_contrib = tsne.fit_transform(train_contribs['Contrib'].apply(pd.Series))
    tsne_contrib = pd.DataFrame(tsne_contrib, columns=['tsne_1', 'tsne_2'])
    
    tsne_flatten = tsne.fit_transform(train_contribs['Flatten'].apply(pd.Series))
    tsne_flatten = pd.DataFrame(tsne_flatten, columns=['tsne_1', 'tsne_2'])
    
    tsne_latent = tsne.fit_transform(train_contribs['Latent'].apply(pd.Series))
    tsne_latent = pd.DataFrame(tsne_latent, columns=['tsne_1', 'tsne_2'])
    
    fig, ax = plt.subplots(3, 1, figsize=(10, 18))
    sns.scatterplot(
        data=tsne_contrib,
        x='tsne_1', y='tsne_2',
        hue=train_contribs['Label'],
        palette='Paired',
        ax=ax[0]
    )
    ax[0].set_title('Contributions')
    ax[0].grid(alpha=0.4)
    
    sns.scatterplot(
        data=tsne_flatten,
        x='tsne_1', y='tsne_2',
        hue=train_contribs['Label'],
        palette='Paired',
        ax=ax[1]
    )
    ax[1].set_title('Flatten')
    ax[1].grid(alpha=0.4)
    
    sns.scatterplot(
        data=tsne_latent,
        x='tsne_1', y='tsne_2',
        hue=train_contribs['Label'],
        palette='Paired',
        ax=ax[2]
    )
    ax[2].set_title('Latent')
    ax[2].grid(alpha=0.4)
    
    plt.show()

In [ ]:
viz_contrib(train_contribs)

In [ ]:
train_contribs.columns

In [ ]:
from torchvision.models import resnet18 as model 
from torchvision.models import ResNet18_Weights as model_weights
# Cargar el modelo preentrenado de DenseNet121
weights = model_weights.IMAGENET1K_V1
model = model(weights=weights)
model.eval()  # Poner el modelo en modo de evaluación


def hook(module, input, output):
    global flatten
    flatten.append(output.clone().detach())
    flatten = [tensor.view(-1) for batch in flatten for tensor in batch]
    
model.layer4[-1].bn2.register_forward_hook(hook)

In [ ]:
def top5_cosine(data, y, index, flatten, contrib, latent, net_label, df): #incluir label original, label del modelo
    
    # Convertir cada array en la columna del DataFrame en un tensor
    contrib_list = [torch.tensor(x) for x in df['Contrib']]
    flatten_list = [torch.tensor(x) for x in df['Flatten']]
    latent_list = [torch.tensor(x) for x in df['Latent']]
    

    # Calcular la similitud del coseno para las contribuciones
    similarities_contrib = [F.cosine_similarity(contrib, tensor, dim=0) for tensor in contrib_list]
    similarities_contrib = torch.tensor(similarities_contrib)
    
    # Calcular la similitud del coseno para los flatten
    similarities_flatten = [F.cosine_similarity(flatten, tensor, dim=0) for tensor in flatten_list]
    similarities_flatten = torch.tensor(similarities_flatten)
    
   # Calcular la similitud del coseno para los flatten
    similarities_latent = [F.cosine_similarity(latent, tensor, dim=0) for tensor in latent_list]
    similarities_latent = torch.tensor(similarities_latent) 

    # Encontrar los top 5
    top5_indices_contrib = torch.topk(similarities_contrib, 5).indices
    top5_indices_flatten = torch.topk(similarities_flatten, 5).indices
    top5_indices_latent = torch.topk(similarities_latent, 5).indices

    # Visualizar la imagen original
    fig, axs = plt.subplots(4, 5, figsize=(15, 10))
    img = torchvision.utils.make_grid(data[index]) / 2 + 0.5
    npimg = img.numpy()
    axs[0, 2].imshow(np.transpose(npimg, (1, 2, 0)))
    axs[0, 2].set_title(f"Real Label: {torch.argmax(y[index])}\n Model Label: {net_label}")

    # Desactivar los espacios vacíos en la primera fila
    for i in [0, 1, 3, 4]:
        axs[0, i].axis("off")

    # Visualizar las imágenes más similares para los tensores
    for i, idx in enumerate(top5_indices_contrib):
        img = torchvision.utils.make_grid(images_sample[idx]) / 2 + 0.5
        npimg = img.numpy()
        axs[1, i].imshow(np.transpose(npimg, (1, 2, 0)))
        axs[1, i].set_title(f"Contrib {i+1} - Sim: {similarities_contrib[idx]:.3f}\n Real Label: {torch.argmax(y_sample[idx])}")

    # Visualizar las imágenes más similares para los flatten
    for i, idx in enumerate(top5_indices_flatten):
        img = torchvision.utils.make_grid(images_sample[idx]) / 2 + 0.5
        npimg = img.numpy()
        axs[2, i].imshow(np.transpose(npimg, (1, 2, 0)))
        axs[2, i].set_title(f"Flatten {i+1} - Sim: {similarities_flatten[idx]:.3f}\n Real Label: {torch.argmax(y_sample[idx])}")

    # Visualizar las imágenes más similares para los flatten
    for i, idx in enumerate(top5_indices_latent):
        img = torchvision.utils.make_grid(images_sample[idx]) / 2 + 0.5
        npimg = img.numpy()
        axs[3, i].imshow(np.transpose(npimg, (1, 2, 0)))
        axs[3, i].set_title(f"Latent {i+1} - Sim: {similarities_latent[idx]:.3f}\n Real Label: {torch.argmax(y_sample[idx])}")
    
    for ax in axs.flat:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def procesar_lista(lista, top_n, positivo=True):
    if positivo:
        elementos = [(i, num) for i, num in enumerate(lista) if num > 0]
    else:
        elementos = [(i, num) for i, num in enumerate(lista) if num < 0]
    total_elementos = sum(num for _, num in elementos)
    porcentajes = [(i, num, num / total_elementos * 100) for i, num in elementos]
    porcentajes_ordenados = sorted(porcentajes, key=lambda x: x[2], reverse=positivo)
    return porcentajes_ordenados[:top_n]

def graficar_elementos(ax, resultado, top_n, positivo=True):
    indices = [tupla[0] for tupla in resultado]
    entradas = [tupla[1] for tupla in resultado]
    porcentajes = [tupla[2] for tupla in resultado]

    color = 'tab:blue' if positivo else 'tab:red'
    ax.barh(range(len(indices)), porcentajes, color = color)

    etiquetas_y = [f"Entrada: {i}, Valor: {e:.3f}" for i, e in zip(indices, entradas)]
    ax.set_yticks(range(len(indices)))
    ax.set_yticklabels(etiquetas_y)

    ax.invert_yaxis()

    ax.set_xlabel('Porcentaje (%)')
    if positivo:
        ax.set_title(f'Top {top_n} Entradas Positivas por Porcentaje')
    else:
        ax.set_title(f'Top {top_n} Entradas Negativas por Porcentaje')

def graficar_entradas(tensor, top_n=15):
    lista = tensor.tolist()
    fig, axs = plt.subplots(1, 2, figsize=(20, 6))

    resultado_positivos = procesar_lista(lista, top_n, positivo=True)
    graficar_elementos(axs[0], resultado_positivos, top_n, positivo=True)

    resultado_negativos = procesar_lista(lista, top_n, positivo=False)
    graficar_elementos(axs[1], resultado_negativos, top_n, positivo=False)

    plt.tight_layout()
    plt.show()

In [ ]:
images = torch.load('images.pth')

In [ ]:
index = 250
fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(images[index].permute(1, 2, 0))
ax.axis('off')
ax.set_title("Real Image")
plt.show()

In [ ]:
flatten = []
with torch.inference_mode():
    net_label = model(images[index].unsqueeze(0))
    net_label = torch.argmax(net_label)
    _, contrib, latent = get_contrib(flatten[0], classifier_big, betas_tensor)
    avgpool = torch.nn.AdaptiveAvgPool2d((1,1))(flatten[0].view(512, 7, 7)).view(-1)
    _, avg_contrib, _ = get_contrib(avgpool, classifier_small, betas_tensor_small)


top5_cosine(images, y, index, flatten[0], contrib, latent[0], net_label, train_contribs)
graficar_entradas(contrib, top_n=10)

In [ ]:
flatten2 = flatten[0].clone()
contrib2 = contrib.clone()
shape = (7, 512, 7)

nobs_cam = flatten2*contrib2
nobs_cam = nobs_cam.view(shape)

nobs_cam_mean = torch.mean(nobs_cam, dim=1)
nobs_cam_mean = (nobs_cam_mean - nobs_cam_mean.min()) / (nobs_cam_mean.max() - nobs_cam_mean.min()) # hacer una prueba con el softmax

nobs_cam_avg = torch.matmul(nobs_cam.permute(0, 2, 1), avg_contrib) / torch.sum(avg_contrib)
nobs_cam_avg = (nobs_cam_avg - nobs_cam_avg.min()) / (nobs_cam_avg.max() - nobs_cam_avg.min())

contrib2 = contrib2.view(shape)
contrib2 = torch.mean(contrib2, dim=1)
contrib2 = (contrib2 - contrib2.min()) / (contrib2.max() - contrib2.min())

flatten2 = flatten2.view(shape)
flatten2 = torch.mean(flatten2, dim=1)
flatten2 = (flatten2 - flatten2.min()) / (flatten2.max() - flatten2.min())

fig, ax = plt.subplots(figsize=(3, 3))
ax.imshow(images[index].permute(1, 2, 0))
ax.axis('off')
ax.set_title("Real Image")
plt.show()

cmap = 'viridis'
fig, ax = plt.subplots(1, 4, figsize=(12, 3))  # Aumentamos el ancho de la figura
ax[0].imshow(flatten2.squeeze(), cmap=cmap)
ax[0].set_title("Flatten")
cbar0 = plt.colorbar(ax[0].imshow(flatten2.squeeze(), cmap=cmap), ax=ax[0])

ax[1].imshow(contrib2.squeeze(), cmap=cmap)
ax[1].set_title("Contrib")
cbar1 = plt.colorbar(ax[1].imshow(contrib2.squeeze(), cmap=cmap), ax=ax[1])

ax[2].imshow(nobs_cam_mean.squeeze(), cmap=cmap)
ax[2].set_title("NObSP CAM")
cbar2 = plt.colorbar(ax[2].imshow(nobs_cam_mean.squeeze(), cmap=cmap), ax=ax[2])

ax[3].imshow(nobs_cam_avg.squeeze(), cmap=cmap)
ax[3].set_title("NObSP CAM - Weight AVG")
cbar2 = plt.colorbar(ax[3].imshow(nobs_cam_avg.squeeze(), cmap=cmap), ax=ax[3])

for ax in ax.flat:
    ax.axis('off')

plt.tight_layout()  # Ajusta el espaciado entre las subtramas
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

pad = (1,)*2
# no rellenar con ceros

nobs_cam_pad = np.pad(nobs_cam_mean, pad_width=(pad, pad), mode='constant', constant_values=0)# no rellenar con ceros 
overlay_mean = to_pil_image(nobs_cam_pad, mode='F').resize((224,224), resample=PIL.Image.BICUBIC)
cmap = colormaps['jet']
overlay_mean = (cmap(np.asarray(overlay_mean) ** 2)[:, :, :3])

input_tensor = images[index].unsqueeze(0)
input_numpy = input_tensor.detach().cpu().numpy()
input_numpy = input_numpy - input_numpy.min()
input_numpy = input_numpy / input_numpy.max()
input_numpy = np.transpose(input_numpy.squeeze(), (1, 2, 0))


ax.imshow(input_numpy)
ax.imshow(overlay_mean, alpha=0.5)
ax.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

pad = (1,)*2
# no rellenar con ceros

nobs_cam_pad = np.pad(nobs_cam_avg, pad_width=(pad, pad), mode='constant', constant_values=0)# no rellenar con ceros 
overlay_avg = to_pil_image(nobs_cam_pad, mode='F').resize((224,224), resample=PIL.Image.BICUBIC)
cmap = colormaps['jet']
overlay_avg = (cmap(np.asarray(overlay_avg) ** 2)[:, :, :3])

input_tensor = images[index].unsqueeze(0)
input_numpy = input_tensor.detach().cpu().numpy()
input_numpy = input_numpy - input_numpy.min()
input_numpy = input_numpy / input_numpy.max()
input_numpy = np.transpose(input_numpy.squeeze(), (1, 2, 0))


ax.imshow(input_numpy)
ax.imshow(overlay_avg, alpha=0.5)
ax.axis('off')
plt.show()

In [ ]:
target_layers = [model.layer4[-1]]
input_tensor = images[index].unsqueeze(0)
cam = GradCAM(model=model, target_layers=target_layers)
targets = [ClassifierOutputTarget(torch.argmax(y[index]))]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Normalizar el tensor a [0, 1] y convertirlo a un array de NumPy
input_numpy = input_tensor.detach().cpu().numpy()
input_numpy = input_numpy - input_numpy.min()
input_numpy = input_numpy / input_numpy.max()

# Ajustar la forma de la máscara para que coincida con la de la imagen
grayscale_cam = grayscale_cam.squeeze()

# Transponer input_numpy para que tenga la forma (3,224,224)
input_numpy = np.transpose(input_numpy.squeeze(), (1, 2, 0))

cam_image = show_cam_on_image(input_numpy, grayscale_cam, use_rgb=True)

fig, ax = plt.subplots(figsize=(5, 5))
#ax.imshow(images[index].permute(1, 2, 0))
ax.imshow(cam_image)
ax.axis('off')
plt.show()


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(input_numpy)
ax[0].imshow(overlay_mean, alpha=0.5)
ax[0].set_title("NObSP CAM")
ax[0].axis('off')

ax[1].imshow(input_numpy)
ax[1].imshow(overlay_avg, alpha=0.5)
ax[1].set_title("NObSP CAM - Weight AVG Contrib")
ax[1].axis('off')

ax[2].imshow(cam_image)
ax[2].axis('off')
ax[2].set_title("GradCAM")

plt.show()